In [3]:
import numpy as np
from skimage import io
from skimage.util import random_noise
from skimage.metrics import peak_signal_noise_ratio
from matplotlib import pyplot as plt
from skimage.metrics import peak_signal_noise_ratio, structural_similarity
from skimage.transform import resize

In [4]:
noisy_img = io.imread ("/Users/jiyoung/Desktop/univ/'23-2/Engineering Design/workspace/dog_noisy.png")
clean_img = io.imread ("/Users/jiyoung/Desktop/univ/'23-2/Engineering Design/workspace/dog.png")

In [11]:
# Median Filter
def median_filter(noisy_img, filter_size=(3, 3), stride=1):
    img_shape = np.shape(noisy_img)
    result_shape = tuple(np.int64((np.array(img_shape[:2]) - np.array(filter_size)) / stride) + 1) + (img_shape[2],)
    result = np.zeros(result_shape)

    for h in range(0, result_shape[0], stride):
        for w in range(0, result_shape[1], stride):
            for c in range(img_shape[2]):
                tmp = noisy_img[h:h + filter_size[0], w:w + filter_size[1], c].ravel()
                tmp = np.sort(tmp)
                result[h, w, c] = tmp[int(len(tmp) / 2)]

    return result

median_filtered_image = median_filter(noisy_img)
# #plt.imsave('median_filtered_image.png', median_filtered_image)
plt.imshow(median_filtered_image)

# median_filtered_img = median_filter(noisy_img)
# median_img_resized = resize ( median_filtered_img, clean_img.shape, mode = 'constant', anti_aliasing=True )
# median_img_resized = np.clip(median_img_resized, 0., 1.0)
# print("Median Filter PSNR: ", peak_signal_noise_ratio(clean_img, median_img_resized))

In [ ]:
median_filtered_img = median_filter(noisy_img)
median_img_resized = resize ( median_filtered_img, clean_img.shape, mode = 'constant', anti_aliasing=True )
median_img_resized = np.clip(median_img_resized, 0., 1.0)


In [4]:
# Gaussian Filter
def gaussian_kernel(k_size, sigma):
    size = k_size//2
    y, x = np.ogrid[-size:size+1, -size:size+1]
    filter = 1/(2*np.pi * (sigma**2)) * np.exp(-1 *(x**2 + y**2) /(2*(sigma**2)))
    sum = filter.sum()
    filter /= sum
    return filter

def padding(img, k_size):
    pad_size = k_size//2
    rows, cols, ch = img.shape
    res = np.zeros((rows + (2*pad_size), cols+(2*pad_size), ch), dtype=np.float)
    if pad_size == 0:
        res = img.copy()
    else:
        res[pad_size:-pad_size, pad_size:-pad_size] = img.copy()
    return res

def gaussian_filtering(img, k_size=5, sigma=4.0):
    rows, cols, channels = img.shape
    filter = gaussian_kernel(k_size, sigma)
    pad_img = padding(img, k_size)
    filtered_img = np.zeros((rows, cols, channels), dtype=np.float32)
    
    for ch in range(0, channels):
        for i in range(rows):
            for j in range(cols):
                filtered_img[i, j, ch] = np.sum(filter * pad_img[i:i+k_size, j:j+k_size, ch])

    return filtered_img

def median_gaussian(noisy_img, clean_img):
    median_img = median_filter(noisy_img)
#    median_img_resized = resize ( median_img, (clean_img.shape[0], clean_img.shape[1]), mode = 'constant', anti_aliasing=True )
    median_img_resized = resize ( median_img, clean_img.shape, mode = 'constant', anti_aliasing=True )
    median_img_resized = np.clip(median_img_resized, 0., 1.0)
    median_gaussian_img = gaussian_filtering(median_img_resized)
    return median_gaussian_img

def bilateral_filter(noisy_img, k_size=5, sigma_space=4, sigma_intensity=0.5):
    h, w, ch = noisy_img.shape
    bilateral_noisy_img = np.zeros((h, w, ch), dtype=np.float32)
    
    for c in range(ch):
        # 공간적인 가우시안 필터
        spatial_filter = gaussian_kernel(k_size, sigma_space)
        
        for i in range(h):
            for j in range(w):
                intensity_center = noisy_img[i, j, c]
                
                weighted_sum = 0.0
                normalization_factor = 0.0
                
                for m in range(-k_size//2, k_size//2 + 1):
                    for n in range(-k_size//2, k_size//2 + 1):
                        i_neighbor = i + m
                        j_neighbor = j + n
                        
                        if 0 <= i_neighbor < h and 0 <= j_neighbor < w:
                            intensity_neighbor = noisy_img[i_neighbor, j_neighbor, c]
                            
                            # 강도 차이에 대한 가중치
                            weight_intensity = np.exp(-(intensity_center - intensity_neighbor)**2 / (2 * sigma_intensity**2))
                            
                            # 공간적인 가우시안 필터 적용
                            weight_spatial = spatial_filter[m + k_size//2, n + k_size//2]
                            
                            # 각 픽셀에 대한 가중치를 누적
                            weight = weight_intensity * weight_spatial
                            weighted_sum += intensity_neighbor * weight
                            normalization_factor += weight
                
                # 정규화된 결과를 저장
                bilateral_noisy_img[i, j, c] = weighted_sum / normalization_factor
    
    return bilateral_noisy_img

# folder_path = "/Users/jiyoung/Desktop/univ/'23-2/Engineering Design/workspace/set1/"

# name = ['baby', 'bagles', 'beach', 'book', 'dog']

# for fn in name:
#     _noisy = '_noisy'
#     png = '.png'

#     filename = fn + png
#     clean_img = io.imread(filename)
#     clean_img = clean_img[:,:,0:3]
#     clean_img = clean_img/255.0

#     filename = ""
#     filename = fn + _noisy + png
#     noisy_img = io.imread(filename)
#     noisy_img = noisy_img[:,:,0:3]
#     noisy_img = noisy_img/255.0

    # median_gaussian_img = median_gaussian(noisy_img, clean_img)
    # title = filename + "denoising\nPSNR: " + str(peak_signal_noise_ratio(clean_img, median_gaussian_img))
    # print(title)

# median_gaussian_img = median_gaussian(noisy_img, clean_img)
# bilateral_median_gaussian_img = bilateral_filter(median_gaussian_img)
# title = "denoising\nPSNR: " + str(peak_signal_noise_ratio(clean_img, bilateral_median_gaussian_img))
# print(title)

# median + bilateral
median_img = median_filter(noisy_img)
bilateral_median_img = bilateral_filter(median_img)
print("PSNR: ", peak_signal_noise_ratio(clean_img, bilateral_median_img))



/var/folders/n1/t3hm6b612gg69mlfjrdv4ht00000gn/T/ipykernel_97422/773462475.py:13: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  res = np.zeros((rows + (2*pad_size), cols+(2*pad_size), ch), dtype=np.float)
